### LDA (based on prev section: k = 6, alpha = 0.9099999999999999 and beta = 0.9099999999999999) + KMeans

In [ ]:
import gensim
import preprocess2
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a Dictionary
id2word = corpora.Dictionary(reviews_docs)

# Create a Corpus
texts = reviews_docs

# Term Document Frequency
corpus = [id2word.doc2bow(doc) for doc in texts]

# Get Document-Word matrix
vectorizer = TfidfVectorizer()
data = vectorizer.fit_transform(reviews_docs_joined).toarray()

In [ ]:
# Get feature names
feature = vectorizer.get_feature_names()

In [ ]:
def get_doc_topic(corpus, model):
    doc_topic = list()
    for doc in corpus:
        doc_topic.append(model.__getitem__(doc, eps=0))
        return doc_topic

In [ ]:
optimal_lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=6, 
                                       random_state=100,
                                       alpha = 0.9099999999999999,
                                       eta = 0.9099999999999999,
                                       per_word_topics=True, 
                                       chunksize=100,
                                       passes=10,)

In [ ]:
lda_data = get_doc_topic(corpus, optimal_lda_model)

In [ ]:
import numpy as np


def gen_init_point(docTopic, docWord, a):

    thershold = int((docTopic.shape[0] // docTopic.shape[1]) + a * docTopic.shape[0])

    # print(thershold)

    topic_mean = docTopic.mean(axis=0)
    # print(topic_mean)
    # print(topic_mean)

    support_doc_n = []
    support_doc_index = []

    for x in range(docTopic.shape[1]):
        topic = docTopic[:, x]
        res_list = topic > topic_mean[x]
        res_index = np.where(res_list == True)
        support_doc_n.append(len(res_index[0]))
        support_doc_index.append(res_index[0])

    # print(support_doc_index)
    # print(support_doc_n)
    support_doc_n = np.array(support_doc_n)
    # print(support_doc_n)
    typical_topic = np.where(support_doc_n > thershold)[0]
    # print(typical_topic)
    # print(typical_topic)

    k_clustering_init = []
    for i in typical_topic:
        # print(i)
        # print(support_doc_index[i])
        # print(dataset[support_doc_index[i]])
        k_clustering_init.append(np.asarray(docWord[support_doc_index[i]].mean(axis=0)).reshape(-1))
    return np.array(k_clustering_init)

In [ ]:
import pandas as pd

def print_top_words(model, feature_names, n_top_words, path):
    print('for reduce dimension')
    out = open(path, 'w')
    # model.components_ = lsa.inverse_transform(model.components_)
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
        out.write(message+'\n')
    out.close()
    print()


def print_cluster(model, feature, n, path):
    print('for cluster')
    out = open(path, 'w')
    for topic_idx, topic in enumerate(model.cluster_centers_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature[i]
                             for i in topic.argsort()[:-n - 1:-1]])
        print(message)
        out.write(message+'\n')
    out.close()
    print()


def output_result(corpus, result, path):
    df = pd.DataFrame({'text': corpus, 'label': result})
    df[['text', 'label']].to_csv(path, index=False)

In [ ]:
from sklearn.cluster import KMeans
from model import gen_init_point
from model import helper
from sklearn.metrics import calinski_harabaz_score
from sklearn.metrics import silhouette_score

def pipeline(model, n_topic, data, a, corpus, feature):
    lda = model(n_components=n_topic)
    data_lda = lda.fit_transform(data)
    n_init_clusters = gen_init_point.gen_init_point(data_lda, data, a)
    # print(n_init_clusters)
    km = KMeans(n_clusters=len(n_init_clusters), init=n_init_clusters)
    km.fit_transform(data)
    print('score')
    print(calinski_harabaz_score(data.toarray(), km.labels_))
    print(silhouette_score(data.toarray(), km.labels_))
    helper.print_top_words(lda, feature, 10, '_topic_word')
    helper.print_cluster(km, feature, 10, '_cluster_meaning_{}')
    helper.output_result(corpus, km.labels_, '_cluster_result_{}')

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation,
n_topic = 20
pipeline.pipeline(LatentDirichletAllocation,n_topic,data,0.1,corpus,feature,'all','lda_chinese')